In [1]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu,SmoothingFunction
import pandas as pd
import numpy as np
import docker
import os
import re
import random
from string import punctuation
import json
import matplotlib.pyplot as plt
punc = punctuation
smooth = SmoothingFunction()

In [2]:
conAst = pd.read_csv('/home/yfliu/paper_data/astResultCsv/contractRes.csv')
conAst['conName'] = conAst['contract_code'].apply(lambda x : x.split(' ')[1].split('{')[0].split('\n')[0])
funAst = pd.read_csv('/home/yfliu/paper_data/astResultCsv/functionRes.csv')
funAst['conName'] = funAst['contract_code'].apply(lambda x : x.split(' ')[1].split('{')[0].split('\n')[0])

In [7]:
temp = set(list(conAst['sni_name'].apply(lambda x : x.split('-')[0])) + list(funAst['sni_name'].apply(lambda x : x.split('-')[0])) )
temp

{'49946',
 '11365',
 '4273',
 '49136',
 '55037',
 '52508',
 '59162',
 '52451',
 '61541',
 '66313',
 '44077',
 '56510',
 '77039',
 '41707',
 '65448',
 '32941',
 '30021',
 '40479',
 '63733',
 '64633',
 '83674',
 '58776',
 '64424',
 '21641',
 '72641',
 '21522',
 '20933',
 '30725',
 '42349',
 '49348',
 '6933',
 '37927',
 '33005',
 '61248',
 '70675',
 '65242',
 '49931',
 '45747',
 '76936',
 '83542',
 '35328',
 '83878',
 '82470',
 '56466',
 '48832',
 '70523',
 '39006',
 '12920',
 '57169',
 '13032',
 '45653',
 '7089',
 '19561',
 '57960',
 '46354',
 '47367',
 '57553',
 '83398',
 '78654',
 '16043',
 '79966',
 '37716',
 '41558',
 '70953',
 '79826',
 '37750',
 '8959',
 '73601',
 '34330',
 '54860',
 '34224',
 '61914',
 '80733',
 '79603',
 '56556',
 '26192',
 '35878',
 '80709',
 '49836',
 '67137',
 '45503',
 '12698',
 '34395',
 '50078',
 '67381',
 '68759',
 '16058',
 '872',
 '49084',
 '13594',
 '25663',
 '10185',
 '24888',
 '33066',
 '77008',
 '20888',
 '65804',
 '51920',
 '9705',
 '3373',
 '62358'

In [9]:
len(list(temp))

1258

In [3]:
ast = conAst

In [4]:
def patternDeal(sol, num):
    if num == 6:
        return [ 
                {
                    'itemName' : item.split(':')[0],
                    'rows' : [ i.strip() for i in item.split(':')[1].split(',') ]
                      }  for item in sol[1:] if ':' in item and 'uint' in item.split(':')[0] and 'uint256' not in item.split(':')[0]
                ]
    return [ 
                {
                    'itemName' : item.split(':')[0],
                    'rows' : [ i.strip() for i in item.split(':')[1].split(',') ]
                      }  for item in sol[1:] if ':' in item 
                ]

#将结果分割，需要不同的split字符串
def getGasResList(address, num):
    pat = open(address,'r').read()
    patList = re.split('\n[0-9]+ : ', pat)#
    ans = {}
    for sol in patList:
        if 'success' not in sol:
            continue
        sol = sol.split('\n')
        sol = sol[:-1]
        if sol[0] in ans.keys():
            ans[sol[0]] += patternDeal(sol, num)
        else:
            ans[sol[0]] = patternDeal(sol, num)
    return ans

#匹配gasPattern与克隆对的范围
def matchRange(item, contract_line):
    return int(item['rows'][0].strip()) >= eval(contract_line)[0] and int(item['rows'][1].strip()) <= eval(contract_line)[1]

#当克隆对合约在patList内，并且范围正确，旧获得
def compareRange(pat, contract_line, name):

    return  [
        item['rows'] for item in pat[name] if matchRange(item, contract_line)
    ] if name in pat.keys() else None

#将gas结果加到bb内
def GasAppendByRange(bb,pat,num):
    bb['pattern{}'.format(num)] = [ compareRange(pat, item[1]['contract_line'], item[1]['code_name'].split('.')[0]) for item in bb.iterrows() ]

def runGasPattern(patNum, rootAddress, bb):
    for num in patNum:
    #切割
        pat = getGasResList(os.path.join(rootAddress,'gasDetectResult{}.txt').format(num), num)
        GasAppendByRange(bb, pat, num)
        
rootAddress = '/home/yfliu/paper_data/gasPattern/new'
patNum = [1,2,3,4,6,7]
runGasPattern(patNum, rootAddress, ast)
ast

,index,sni_index,code_index,code_name,dealed_code,sni_name,contract_code,sni_code,contract_line,name,...,slither_analysis,slither_analysis_not_info,mythril_analysis_not_info,conName,pattern1,pattern2,pattern3,pattern4,pattern6,pattern7
0,0,2,46,0xcf2b19384ea3f027f8620db3f797ae61bb475125.sol,"['contract', 'multisig', 'event', 'Depositaddr...",7089-1.sol,contract multisig {\n\n\n\n \n\n\n\n \n\...,contract multisig { event Deposit(address from...,"[570, 610]",0xcf2b19384ea3f027f8620db3f797ae61bb475125,...,[],[],[],multisig,None,None,None,None,None,None
1,1,8,993,0x002842529757eab873cce9c251087e1b993f9200.sol,"['contract', 'StupidToken', 'function', 'trans...",37927-0.sol,contract StupidToken {\n function transferF...,contract MyToken {\n function transferF...,"[33, 35]",0x002842529757eab873cce9c251087e1b993f9200,...,[],[],[],StupidToken,None,None,None,None,[],None
2,2,6238,993,0x002842529757eab873cce9c251087e1b993f9200.sol,"['contract', 'StupidToken', 'function', 'trans...",37858-0-0.sol,contract StupidToken {\n function transferF...,contract TheTokenContract {\n \n functio...,"[33, 35]",0x002842529757eab873cce9c251087e1b993f9200,...,[],[],[],StupidToken,None,None,None,None,[],None
3,3,29,23020,0x59f243d056ada87eb563c2d507fe4822f8ccccac.sol,"['contract', 'MSD4', 'function', 'greenOnFanCh...",84331-2.sol,contract MSD4 {\n\n \n\n function greenOn...,contract Tested1 {\n function func() public...,"[36, 46]",0x59f243d056ada87eb563c2d507fe4822f8ccccac,...,[],[],[],MSD4,None,None,None,None,[],None
4,4,1745,23020,0x59f243d056ada87eb563c2d507fe4822f8ccccac.sol,"['contract', 'MSD4', 'function', 'greenOnFanCh...",84331-0.sol,contract MSD4 {\n\n \n\n function greenOn...,contract Tested {\n function func() public ...,"[36, 46]",0x59f243d056ada87eb563c2d507fe4822f8ccccac,...,[],[],[],MSD4,None,None,None,None,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,1115,7049,2076,0x4333defa1bf3b49246666d1dea395f98e816a032.sol,"['contract', 'BitQueen', 'is', 'ERC20', 'strin...",66012-0-0.sol,contract BitQueen is ERC20 {\n\n string publi...,contract SimpleToken is ERC20 {\n\n string pu...,"[310, 328]",0x4333defa1bf3b49246666d1dea395f98e816a032,...,[],[],[],BitQueen,None,None,None,None,"[[317, 317]]",None
1116,1116,7060,30455,0xf4c13de243a7716fb7351736228c8337f460d760.sol,"['contract', 'DreamCarToken', 'function', 'get...",71855-1-0.sol,contract DreamCarToken {\n\n function getFo...,contract A {\n function someMethod(address ...,"[16, 20]",0xf4c13de243a7716fb7351736228c8337f460d760,...,[],[],[],DreamCarToken,None,None,[],None,[],None
1117,1117,7082,3571,0x5f002910d3e1bdcdcef62cbcc2b294618e9f7fad.sol,"['contract', 'TokenPriveProviderInterface', 'f...",20864-1-0.sol,contract TokenPriveProviderInterface {\n\n ...,contract Storage {\n function getEntityCoun...,"[86, 90]",0x5f002910d3e1bdcdcef62cbcc2b294618e9f7fad,...,[],[],[],TokenPriveProviderInterface,None,None,None,None,"[[89, 89]]",None
1118,1118,7093,459,0xc849a5fff9befa297d1c3a772bafac9f3c9004ec.sol,"['contract', 'ERC223ReceivingContract', 'funct...",34224-0-0.sol,contract ERC223ReceivingContract {\n\n\n\n ...,contract ERC223Receiver\n{\n function token...,"[38, 56]",0xc849a5fff9befa297d1c3a772bafac9f3c9004ec,...,[],[],[],ERC223ReceivingContract,None,None,None,None,"[[55, 55]]",None


In [32]:
def getRandom(ast):
    gasRan = ast[ast.pattern1.apply(lambda x : x != None) | ast.pattern2.apply(lambda x : x != None) | ast.pattern3.apply(lambda x : x != None) |ast.pattern4.apply(lambda x : x != None) |ast.pattern6.apply(lambda x : x != None) |ast.pattern7.apply(lambda x : x != None)].sample(100)
    bugRan = ast[ast.osiris_analysis.apply(lambda x : x != []) | ast.mythril_analysis_not_info.apply(lambda x : x != []) | ast.slither_analysis_not_info.apply(lambda x : x != [])].sample(100)
    return gasRan, bugRan
gas, bug = getRandom(ast)

def getLookList(ast, flag):
    res = []
    bugLoc = []
    if flag == 'gas':
        bugLoc = ['pattern1','pattern2','pattern3','pattern4','pattern6','pattern7',]
    else:
        bugLoc = ['mythril_analysis_not_info','slither_analysis_not_info','osiris_analysis']
    for idx, row in ast.iterrows():
        sniCode = row['sni_code']
        conCode = row['contract_code']
        text = ''
        for item in bugLoc:
            bug = row[item]
            if type(bug) != float and bug != None and bug != '[]':
                if type(bug) == list and len(bug) == 0:
                    continue
                text = item + '\n' + text + str(bug)
                text += '\n----------------------\n'
        if text == '':
            continue
        text += '\n=====================\n'
        text += sniCode
        text += '\n=====================\n'
        text += conCode
        res.append(text)
    return res


In [33]:
res = getLookList(bug,'bug')
res

["slither_analysis_not_info\nmythril_analysis_not_info\n[{'function': 'sendCurrentPayment()', 'lineno': 189, 'title': 'Integer Underflow', 'type': 'Warning'}, {'function': 'sendCurrentPayment()', 'lineno': 203, 'title': 'State change after external call', 'type': 'Warning'}, {'function': 'sendCurrentPayment()', 'lineno': 205, 'title': 'State change after external call', 'type': 'Warning'}]\n----------------------\n[{'title': 'locked-ether', 'function': 'fallback', 'lineno': [143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153], 'type': 'Medium'}, {'title': 'reentrancy-no-eth', 'function': 'sendCurrentPayment', 'lineno': [187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217], 'type': 'Medium'}, {'title': 'reentrancy-no-eth', 'function': 'sendCurrentPayment', 'lineno': [187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210

In [29]:
num = 0

In [49]:
print(res[num])
num += 1

osiris_analysis
slither_analysis_not_info
[{'title': 'reentrancy-benign', 'function': 'buyTokens', 'lineno': [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'type': 'Low'}]
----------------------
[{'title': 'overflow_bugs', 'lineno': 55, 'function': 'DappTokenSale'}]
----------------------

contract AjTokenSale {
address admin;
AjToken public tokenContract;
uint256 public tokenPrice;
uint256 public tokensSold;

event Sell(address _buyer, uint256 _amount);

function AjTokenSale(AjToken _tokenContract, uint256 _tokenPrice) public {
    
    admin = msg.sender;
    
    tokenContract = _tokenContract;
    
    tokenPrice = _tokenPrice;
}    



function multiply(uint x, uint y) internal pure returns(uint z) {

    require (y == 0 || (z = x * y)/y == x);

}


function buyTokens(uint256 _numberOfTokens) public payable {
    

    require(msg.value == multiply(_numberOfTokens , tokenPrice));

    

    require(tokenContract.balanceOf(this) >= _nu